In [60]:
import os
import pandas as pd
import numpy as np
from skimage import color
from PIL import Image
from skimage.color import rgb2lab, lab2xyz
from PIL import Image, ImageCms
import tempfile

def lab_to_xyz(lab):
    lab = np.array(lab, dtype=float)
    return lab2xyz(lab)

In [61]:
from src.calibration_analyzer.analyzer import AppliedImageAnalyzer, IT8Analyzer
import pandas as pd

In [62]:
analyzer = AppliedImageAnalyzer("Applied Image Slide.tif")
ai_processed = analyzer.process()
pd.DataFrame(ai_processed)['delta'].mean()

np.float64(16.0552)

In [63]:
analyzer.bounding_boxes

{'CA': (314, 323, 79, 88),
 'A1': (69, 78, 318, 327),
 'A2': (69, 78, 526, 535),
 'A3': (69, 78, 734, 743),
 'A4': (69, 78, 942, 951),
 'A5': (69, 78, 1150, 1159),
 'A6': (69, 78, 1358, 1367),
 'B1': (234, 243, 318, 327),
 'B2': (234, 243, 526, 535),
 'B3': (234, 243, 734, 743),
 'B4': (234, 243, 942, 951),
 'B5': (234, 243, 1150, 1159),
 'B6': (234, 243, 1358, 1367),
 'C1': (399, 408, 318, 327),
 'C2': (399, 408, 526, 535),
 'C3': (399, 408, 734, 743),
 'C4': (399, 408, 942, 951),
 'C5': (399, 408, 1150, 1159),
 'C6': (399, 408, 1358, 1367),
 'D1': (564, 573, 318, 327),
 'D2': (564, 573, 526, 535),
 'D3': (564, 573, 734, 743),
 'D4': (564, 573, 942, 951),
 'D5': (564, 573, 1150, 1159),
 'D6': (564, 573, 1358, 1367)}

In [4]:
analyzer.get_avg_color("CA")

array([189, 191, 192])

In [64]:
analyzer2 = IT8Analyzer("IT8 E130102.tif", "IT8 Reference File E130102.txt")
it8_processed = analyzer2.process()
pd.DataFrame(it8_processed)['delta'].mean()

np.float64(13.744826388888889)

In [13]:
import datetime
import numpy as np

def generate_ti3_from_analyzer_xyz(
    ai_processed,
    calibration_file,
    out_path,
    descriptor="Applied Image Calibration"
):
    obs_by_label = {p["label"]: p for p in ai_processed}

    labels = [
        lbl for lbl in obs_by_label
        if lbl in calibration_file and "lab" in calibration_file[lbl]
    ]

    calibration_file["__WHITE__"] = {
        "lab": [100.0, 0.0, 0.0]
    }

    now = datetime.datetime.now().strftime("%Y-%m-%d")

    with open(out_path, "w") as f:
        f.write("CTI3\n\n")
        f.write(f'DESCRIPTOR "{descriptor}"\n')
        f.write('ORIGINATOR "Python Analyzer"\n')
        f.write(f'CREATED "{now}"\n')
        f.write('DEVICE_CLASS "INPUT"\n')
        f.write('COLOR_REP "RGB_XYZ"\n\n')

        f.write("NUMBER_OF_FIELDS 7\n")
        f.write("BEGIN_DATA_FORMAT\n")
        f.write("SAMPLE_ID RGB_R RGB_G RGB_B XYZ_X XYZ_Y XYZ_Z\n")
        f.write("END_DATA_FORMAT\n\n")

        f.write(f"NUMBER_OF_SETS {len(labels)+1}\n")
        f.write("BEGIN_DATA\n")

        for idx, label in enumerate(labels, start=1):
            obs = obs_by_label[label]
            lab = np.array(calibration_file[label]["lab"], dtype=float)
            xyz = lab_to_xyz(lab)

            # Clamp RGB
            r = max(0, min(255, obs["R"]))
            g = max(0, min(255, obs["G"]))
            b = max(0, min(255, obs["B"]))

            f.write(
                f"{label} "
                f"{r:.0f} {g:.0f} {b:.0f} "
                f"{xyz[0]:.6f} {xyz[1]:.6f} {xyz[2]:.6f}\n"
            )
        
        f.write(
            "__WHITE__ "
            "255 255 255 "
            "95.047000 100.000000 108.883000\n"
        )

        f.write("END_DATA\n")

In [14]:
generate_ti3_from_analyzer_xyz(ai_processed, analyzer.calibration_file, "temp.txt")

In [15]:
!argyll/Argyll_V3.4.1/bin/txt2ti3 temp.txt temp

In [16]:
!argyll/Argyll_V3.4.1/bin/colprof temp

argyll/Argyll_V3.4.1/bin/colprof: Warning - xfit_fit: Conjgrad failed to converge, residual error = 0.130412
calc_ocent failed to return in-gamut focal point!
